<a href="https://colab.research.google.com/github/NUELBUNDI/PDS_PROJECT/blob/main/mpesa_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import io
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
url="https://raw.githubusercontent.com/NUELBUNDI/PDS_PROJECT/main/mpesa_payment.csv"
df=pd.read_csv(url,encoding = "ISO-8859-1")

# df.head(1)

In [4]:
df.to_csv("data.csv")

In [5]:
# Clean Data

class  clean_data:

  def __init__(self,data_frame):
    self.data_frame=data_frame

  def clean_data(self):
      # rename the columns name
      col_name=[]
      new_colname=[]
      for colname in self.data_frame.columns:
        col_name.append(colname)
        output=colname.replace(' ','').replace('.','').lower()
        new_colname.append(output)
      col_rename_dict={col_name[i]:new_colname[i] for i in range(len(new_colname))}
      self.data_frame.rename(columns=col_rename_dict,inplace=True)
      # Subset the data
      df2=self.data_frame[['postingdate','sourcecode','documentno','externaldocumentno','amount','businessunitcode','departmentcode','balaccountno','description','userid']]

      return df2                     

obj=clean_data(df)

df2=obj.clean_data()


In [ ]:
def convert_amount(val):
  res=val.replace(',','').replace('-','')
  return res
df2['amount']=df2['amount'].apply(lambda x :convert_amount(x)).astype('float')
df=df2

df.head(2)


In [7]:
pd.set_option('mode.chained_assignment', None)

df['businessunitcode']=df['businessunitcode'].astype('str')
df['departmentcode']=df['departmentcode'].astype('str')

In [8]:
# Drop all reversed Entries

df_reversed=df[df['sourcecode']=='REVERSAL']

doc_no_reversed=[]
for x in df_reversed['documentno']:
  doc_no_reversed.append(x)

rows_to_drop=df.query('documentno==@doc_no_reversed')
indexName=rows_to_drop.index
df.drop(indexName, inplace=True)

df['sourcecode'].unique()


array(['GENJNL', 'PAYMENTJNL', 'JOBGLJNL', 'CASHRECJNL'], dtype=object)

In [9]:
# Drop all sweep posted to that account

drop_mpesa_sweep=df.query('balaccountno=="B302010*" | balaccountno=="B301010"').index
df.drop(drop_mpesa_sweep,inplace=True)

In [10]:
# Categorise the Document No.

df['type']=df['documentno'].str[:3]

for x in df['type'].unique():
    print(x,end=',')


MED,PC0,FLT,CRE,GJN,JJN,R00,

In [19]:
# Casual Payment
df_casuals=df.query('balaccountno=="P100500" & type=="GJN"')
mask1=df_casuals['description'].str.contains('casual',case=False,na=False)
mask2=df_casuals['description'].str.contains('Casual Pay',case=False,na=False)
mask3 =df_casuals['description'].str.contains('Labour',case=False,na=False)

df_casuls=df_casuals[mask1 | mask2 | mask3].head(2)


In [27]:
# Subset other categories of payments.

df_salary=df.query('balaccountno=="C200010" & type=="GJN"')
df_medical=df.query('type=="MED"')
df_pettycash=df.query('type=="PC0"')
df_float=df.query('type=="FLT"')
df_customer_refund=df.query('type=="CRE"')


In [30]:
#Import the files to excel


#create a Pandas Excel writer using XlsxWriter as the engine
writer=pd.ExcelWriter('B330020.xlsx',engine='xlsxwriter')

#write each DataFrame to a specific sheet
df_casuals.to_excel(writer,sheet_name='Casuals',index=False)
df_salary.to_excel(writer,sheet_name='Salary',index=False)
df_medical.to_excel(writer,sheet_name='Medical',index=False)
df_pettycash.to_excel(writer,sheet_name='Pettycash',index=False)
df_float.to_excel(writer,sheet_name='Float',index=False)
df_customer_refund.to_excel(writer,sheet_name='Customer Refund',index=False)

#close the Pandas Excel writer and output the Excel file
writer.save()
